In [1]:
import os
import pandas as pd
import numpy as np
import re
import math
from datetime import datetime as dt
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import scipy.stats
import statsmodels.api as sm
from statsmodels.graphics.regressionplots import add_lowess
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [2]:
os.getcwd()

'C:\\Users\\megha\\Desktop\\IIIDS'

In [3]:
#reading the csv file in a dataframe
conviva = pd.read_csv('conviva_sampledata.csv')

In [4]:
conviva

,viewerid,asset,deviceos,country,state,city,asn,isp,starttime,startuptime,...,averagebitrate,startuperror,ipaddress,cdn,sessiontag_c3_player_name,sessiontag_c3_client_brand,sessiontag_connectiontype,sessiontag_c3_client_hwtype,sessiontag_accesstype,churn
0,0171fdb9-d8db-4e00-a9d8-c1619811ac66,[0-0-27132] Kedarnath,Android,india,madhya pradesh,bhopal,24560,"ASN: AIRTELBROADBAND-AS-AP Bharti Airtel Ltd.,...",1559557555,5794,...,0,0,122.168.85.95,AKAMAI,Z5+FireTV+Jwplayer,Xiaomi,Data,TV,Premium,1
1,0171fdb9-d8db-4e00-a9d8-c1619811ac66,[0-0-35166] Simmba,Android,india,madhya pradesh,bhopal,24560,"ASN: AIRTELBROADBAND-AS-AP Bharti Airtel Ltd.,...",1559557880,3804,...,0,0,122.168.85.95,AMAZON,Z5+FireTV+Jwplayer,Xiaomi,Data,TV,Premium,1
2,0171fdb9-d8db-4e00-a9d8-c1619811ac66,[0-0-35021] Nirahua Hindustani 3,Android,india,madhya pradesh,indore,24560,"ASN: AIRTELBROADBAND-AS-AP Bharti Airtel Ltd.,...",1559448731,15589,...,0,0,122.170.203.36,AKAMAI,Z5+FireTV+Jwplayer,Xiaomi,Data,TV,Premium,1
3,02d7f343-f75d-4f10-a5c4-7f21ed6ff341,[0-1-215897] Skyfire - Episode 8 - The Monster,Android,india,uttar pradesh,ghaziabad,134375,ASN: FWSPL-AS-AP Fusionnet Web Services Privat...,1561486982,9716,...,0,0,103.219.230.183,AKAMAI,Z5+Android+Player_3.4.3,Oppo,WiFi,Mobile+Phone,Premium,1
4,02d7f343-f75d-4f10-a5c4-7f21ed6ff341,[0-0-6956] Great Grand Masti,Android,india,uttar pradesh,ghaziabad,134375,ASN: FWSPL-AS-AP Fusionnet Web Services Privat...,1561834149,4567,...,0,0,103.219.229.92,AKAMAI,Z5+Android+Player_3.4.3,Oppo,WiFi,Mobile+Phone,Premium,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,7317642d-2608-48ff-b938-c0766b4b3447,[0-1-224908] Akka Chellellu - Akka Chellellu -...,Android,india,karnataka,gundlupet,9829,"ASN: BSNL-NIB National Internet Backbone, IN",1560189704,-1,...,0,0,117.246.121.126,TATA,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0
1048571,7317642d-2608-48ff-b938-c0766b4b3447,[0-1-224866] Ganga Manga - Manga devises a pla...,Android,india,delhi,sansad marg,9829,"ASN: BSNL-NIB National Internet Backbone, IN",1560153864,4083,...,0,0,117.209.255.97,TATA,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0
1048572,7317642d-2608-48ff-b938-c0766b4b3447,[0-1-220446] Suryavamsham - With ulterior moti...,Android,india,tamil nadu,teynampet,9829,"ASN: BSNL-NIB National Internet Backbone, IN",1559373793,3266,...,0,0,117.246.253.251,AKAMAI,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0
1048573,7317642d-2608-48ff-b938-c0766b4b3447,[0-1-230078] Muddha Mandaram - Muddha Mandaram...,Android,india,karnataka,gundlupet,9829,"ASN: BSNL-NIB National Internet Backbone, IN",1561073056,2264,...,0,0,117.249.225.150,AMAZON,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0


<h1><b>Q1 How many users watched Kaafir?</b></h1>

In [5]:
#selected only those records from the dataframe which do not contain NA values
conviva_notna = conviva[conviva['asset'].notna()]

In [6]:
conviva_notna.shape

(1047283, 23)

In [7]:
#selected those records from the conviva_notna dataframe whose asset column entry contains the substring "Kaafir"
kaafir_viewers = conviva_notna[conviva_notna['asset'].str.contains('Kaafir')]

In [8]:
#unique count of viewerids from the kaafir_viewers series
kaafir_unique_viewers = kaafir_viewers['viewerid'].nunique()

In [9]:
kaafir_unique_viewers

13783

<h1><b>Q2 Those who have watched Kaafir, what’s per viewer average time spent on it?</b></h1> 

In [10]:
kaafir_per_viewer_playtime = kaafir_viewers['playingtime'].sum()

In [11]:
#calculating the average per viewer time and dividing it by 60000 to convert the milliseconds into minutes
kaafir_per_viewer_avg = (kaafir_per_viewer_playtime / kaafir_unique_viewers) / 60000

In [12]:
print("The average per viewer time spent on watching Kaafir is",math.ceil(kaafir_per_viewer_avg),"minutes.")

The average per viewer time spent on watching Kaafir is 40 minutes.


<h1><b>Q3 How many users spent more than 2 Hrs watching kaafir on any day?</b></h1> 

In [13]:
#converting the starttime from UNIX time to datetime format in the table of Kaafir viewers
kaafir_viewers['starttime'] = pd.to_datetime(kaafir_viewers['starttime'],unit='s')

#converting datetime objects to just dates
kaafir_viewers['starttimejustdate'] = kaafir_viewers['starttime'].dt.date

In [14]:
#grouping the kaafir viewers data on viewerid and view dates, and calculating the total time spent viewing kaafir that day
kaafir_viewers_by_date = kaafir_viewers.groupby(['viewerid','starttimejustdate'])['playingtime'].sum()

In [15]:
kaafir_viewers_by_date.shape

(23926,)

In [16]:
kaafir_viewers_by_date = kaafir_viewers_by_date.reset_index()

In [17]:
#selecting those entries where the viewing time has exceeded 2 hours
kaafir_grthan_2hr = kaafir_viewers_by_date[kaafir_viewers_by_date['playingtime'] > 7.2e+6]

In [18]:
kaafir_grthan_2hr.shape

(689, 3)

In [19]:
#number of records with playing time greater than 2 hours
kaafir_grthan_2hr.shape

(689, 3)

In [20]:
#unique number of viewers who have watched kaafir for more than 2 hours on any day
kaafir_grthan_2hr_unique = kaafir_grthan_2hr['viewerid'].nunique()

In [21]:
print(kaafir_grthan_2hr_unique,"users spent more than 2 hours watching Kaafir on any day.")

679 users spent more than 2 hours watching Kaafir on any day.


<h1><b>Q4 How many of users watched kaafir more than 50% of the duration?</b></h1>

In [22]:
#finding out duration of Kaafir episodes
#reading tv_cms data in a pandas dataframe
tv_cms = pd.read_csv('tv_cms_06092019_without_genre.csv')

In [23]:
#isolating the episodes from other show types
kaafir_duration_episode = tv_cms[tv_cms['title'].str.contains('Kaafir') & tv_cms['Subtype'].str.contains('episode')]

In [24]:
kaafir_duration_episode

,Unnamed: 0,Id_tv,DateCreated,Index,Duration,seasonid,episode_id,DateModified,DatePublished,PublicationState,...,show_id_episodetable,title,AudioLanguages,show_id_showtable,title_showtable,OriginalExternalId,SubtitleLanguages,subtype_tvshow,Rating,businessType
1217,1217,4a92c371-5ec6-4b96-bfef-65f1b874efa3,08-08-2019 07:40:20 +00:00,4,2848.0,0-2-1791,0-1-252423,30-08-2019 12:43:43 +00:00,30-08-2019 13:02:57 +00:00,2,...,0-6-1985,Kaafir,kn,0-6-1985,Kaafir,NaN,en,original,5.0,premium_downloadable
3781,3781,d49ab073-05dd-4cc3-98c3-d3144eccf146,08-08-2019 07:58:13 +00:00,5,2749.0,0-2-1791,0-1-252439,30-08-2019 12:46:40 +00:00,30-08-2019 13:03:00 +00:00,2,...,0-6-1985,Kaafir,kn,0-6-1985,Kaafir,NaN,en,original,5.0,premium_downloadable
23828,23828,fd69357c-0c43-4d2d-bca7-39c26b7f0faa,12-06-2019 10:44:44 +00:00,2,2370.0,0-2-1573,0-1-226365,28-08-2019 08:48:25 +00:00,28-08-2019 08:48:48 +00:00,3,...,0-6-1741,Kaafir,hi,0-6-1741,Kaafir,1741,NaN,original,5.0,premium_downloadable
25434,25434,4b9f13b4-d127-4bbf-90a7-5f5ba4b54d63,08-08-2019 07:01:31 +00:00,2,2370.0,0-2-1792,0-1-252374,30-08-2019 12:14:44 +00:00,30-08-2019 12:34:25 +00:00,2,...,0-6-1986,Kaafir,ml,0-6-1986,Kaafir,NaN,en,original,5.0,premium_downloadable
27844,27844,38520534-ca5a-4556-ba95-7603cf5e70eb,08-08-2019 09:12:37 +00:00,6,2804.0,0-2-1790,0-1-252470,30-08-2019 11:57:41 +00:00,30-08-2019 12:05:48 +00:00,2,...,0-6-1984,Kaafir,te,0-6-1984,Kaafir,NaN,en,original,5.0,premium_downloadable
35271,35271,248c5746-652c-4ec5-b2a6-c28460e670d2,12-06-2019 10:58:39 +00:00,6,2804.0,0-2-1573,0-1-226388,28-08-2019 10:04:24 +00:00,28-08-2019 10:05:48 +00:00,3,...,0-6-1741,Kaafir,hi,0-6-1741,Kaafir,1741,NaN,original,5.0,premium_downloadable
40536,40536,c37487ac-6c41-4906-880e-61c46469cf43,08-08-2019 07:01:29 +00:00,2,2370.0,0-2-1791,0-1-252373,30-08-2019 12:40:15 +00:00,30-08-2019 13:03:00 +00:00,2,...,0-6-1985,Kaafir,kn,0-6-1985,Kaafir,NaN,en,original,5.0,premium_downloadable
40731,40731,442ab921-6361-4035-a09d-c301b741557d,12-06-2019 10:42:48 +00:00,1,1974.0,0-2-1573,0-1-226358,28-08-2019 09:44:17 +00:00,28-08-2019 09:49:08 +00:00,3,...,0-6-1741,Kaafir,hi,0-6-1741,Kaafir,1741,NaN,original,5.0,premium_downloadable
47394,47394,c39f89ad-a0f1-4735-a04e-0faaa330a389,08-08-2019 07:58:08 +00:00,5,2749.0,0-2-1789,0-1-252437,30-08-2019 09:48:24 +00:00,30-08-2019 10:02:56 +00:00,2,...,0-6-1983,Kaafir,ta,0-6-1983,Kaafir,NaN,en,original,5.0,premium_downloadable
52661,52661,d1ae62c9-00aa-4914-a601-96e93356dedb,08-08-2019 07:40:18 +00:00,4,2848.0,0-2-1790,0-1-252422,30-08-2019 11:53:10 +00:00,30-08-2019 12:05:50 +00:00,2,...,0-6-1984,Kaafir,te,0-6-1984,Kaafir,NaN,en,original,5.0,premium_downloadable


In [25]:
#episode language assumed to be Hindi for analytical purposes
kaafir_duration_episode = kaafir_duration_episode[kaafir_duration_episode['AudioLanguages'].str.contains('hi')]

In [26]:
#total duration of all Kaafir episodes
kaafir_duration_episode['Duration'].sum()

20440.0

In [27]:
#50% duration of Kaafir
kaafir_duration_50per = kaafir_duration_episode['Duration'].sum() / 2

In [28]:
#converting the 50% duration of Kaafir from seconds to milliseconds
kaafir_duration_50per = kaafir_duration_50per * 1000

In [29]:
#finding out how many viewerids have playing time greater than 50%
kaafir_viewers_gr50per = kaafir_viewers.groupby('viewerid')['playingtime'].sum()

In [30]:
kaafir_viewers_gr50per = kaafir_viewers_gr50per.reset_index()

In [31]:
kaafir_viewers_gr50per = kaafir_viewers_gr50per[kaafir_viewers_gr50per['playingtime'] > kaafir_duration_50per]

In [32]:
kaafir_viewers_gr50per.shape

(475, 2)

In [33]:
print(len(kaafir_viewers_gr50per),"users watched Kaafir for more than 50% of the duration.")

475 users watched Kaafir for more than 50% of the duration.


<h1><b>Q10 What % of the Kaafir viewers churned in next month?</b></h1>

In [34]:
#dropped duplicate records of the viewers who watched Kaafir
kaafir_churners = kaafir_viewers[['viewerid','churn']].drop_duplicates()

In [35]:
#calculated the total sum of the churn column of the unique viewers to find out how many viewers churned
kaafir_actually_churned = kaafir_churners['churn'].sum()

In [36]:
#divied sum of churners with the count of the actual number of Kaafir viewers
kaafir_churn_percentage = (kaafir_actually_churned / kaafir_unique_viewers) * 100

In [37]:
print(math.ceil(kaafir_churn_percentage),"% of Kaafir viewers churned next month")

46 % of Kaafir viewers churned next month


<h1><b>Q11 What % of the subscriber base churned in next month? </b></h1>

In [38]:
#selected the unique viewerid and the churn from the conviva table 
subscriber_base = conviva[['viewerid','churn']].drop_duplicates()

In [39]:
subscriber_base_count = subscriber_base['viewerid'].count()

In [40]:
subscriber_base_count

100532

In [41]:
#calculated the total sum of the churn column of the unique viewers to find out how many viewers churned
subscriber_churn_count = subscriber_base['churn'].sum()

In [42]:
subscriber_churn_count

62948

In [43]:
#divied sum of churners with the count of the actual number of subscribers
subscriber_churn_percentage = (subscriber_churn_count / subscriber_base_count)*100

In [44]:
print(math.ceil(subscriber_churn_percentage),"% of subscriber base churned next month")

63 % of subscriber base churned next month


<h1><b>Q5 How many watched All episodes of Kaafir (irrespective of how much)? </b></h1>

In [45]:
#finding the episode count of Kaafir (assuming the language to be Hindi)
tv_cms_kaafir = tv_cms[tv_cms['title'].str.contains('Kaafir')]

In [46]:
tv_cms_kaafir.shape

(56, 22)

In [47]:
#dropping records with null values
tv_cms_kaafir.dropna()

,Unnamed: 0,Id_tv,DateCreated,Index,Duration,seasonid,episode_id,DateModified,DatePublished,PublicationState,...,show_id_episodetable,title,AudioLanguages,show_id_showtable,title_showtable,OriginalExternalId,SubtitleLanguages,subtype_tvshow,Rating,businessType


In [48]:
#selecting only those Kaafir entries which are episodes
tv_cms_kaafir_episodes = tv_cms_kaafir[tv_cms_kaafir['Subtype'].str.contains('episode')]

In [49]:
#language assumed for this analysis is Hindi
tv_cms_kaafir_episodes_hindi = tv_cms_kaafir_episodes[tv_cms_kaafir_episodes['AudioLanguages'].str.contains('hi')]

In [50]:
#number of Kaafir episodes in total, in Hindi
tv_cms_kaafir_episodes_hindi.shape

(8, 22)

In [51]:
#total entries in conviva with Kaafir as asset
kaafir_viewers.shape

(53245, 24)

In [52]:
#dropping entries with null values and selecting their viewerids and assets only
kuid_unique = kaafir_viewers[['viewerid','asset']].drop_duplicates() 

In [53]:
#selecting those entries which were episodes only
kuid_unique_episodes = kuid_unique[kuid_unique['asset'].str.contains('Episode')]

In [54]:
#mumber of episodes Kaafir watchers have viewed
kaafir_episodes = kuid_unique_episodes['viewerid'].value_counts()

In [55]:
#selecting only those viewer entries who watched all 8 episodes
kaafir_all_watchers = kaafir_episodes[kaafir_episodes.values >= 8]

In [56]:
print(kaafir_all_watchers.size,"users watched all episodes of Kaafir")

132 users watched all episodes of Kaafir


<h1><b>Q9 How many users watched kaafir trailer and then went to watch kaafir show? </b></h1>

In [57]:
#Kaafir viewers who have watched the trailer
kaafir_trailer_viewers = kaafir_viewers[kaafir_viewers['asset'].str.contains('Trailer')][['viewerid','starttime']]

In [58]:
#Kaafir viewers who have watched the episodes
kaafir_episode_viewers = kaafir_viewers[kaafir_viewers['asset'].str.contains('Episode')][['viewerid','starttime']]

In [59]:
#renaming the starttime column of the trailer watchers table to trailerstarttime
kaafir_trailer_viewers.rename(columns={"starttime": "trailerstarttime"},inplace = True)

In [60]:
#renaming the starttime column of the episode watchers table to episodestarttime
kaafir_episode_viewers.rename(columns={"starttime": "episodestarttime"},inplace = True)

In [61]:
#count of trailer viewers
kaafir_trailer_viewers.shape

(3276, 2)

In [62]:
#count of episode viewers
kaafir_episode_viewers.shape

(48059, 2)

In [63]:
#inner join of the tables containing trailer and episode watchers on viewerid
kaafir_trailer_episode_viewers = pd.merge(kaafir_episode_viewers, kaafir_trailer_viewers, how='inner', on=['viewerid'])

In [64]:
#count of trailer and episode viewers
kaafir_trailer_episode_viewers

,viewerid,episodestarttime,trailerstarttime
0,b9e7c6dd-16b7-48da-926f-d0ce84415e62,2019-06-17 11:21:19,2019-06-03 18:12:46
1,b9e7c6dd-16b7-48da-926f-d0ce84415e62,2019-06-17 14:56:02,2019-06-03 18:12:46
2,b9e7c6dd-16b7-48da-926f-d0ce84415e62,2019-06-17 12:26:22,2019-06-03 18:12:46
3,81614ef4-d962-43c2-9b44-5fa1cb31e69b,2019-06-21 15:10:45,2019-06-10 10:29:35
4,81614ef4-d962-43c2-9b44-5fa1cb31e69b,2019-06-21 15:13:26,2019-06-10 10:29:35
...,...,...,...
10149,0c4dee4e-4873-4f69-b4d0-c57baa642b77,2019-06-15 02:41:36,2019-06-06 04:12:25
10150,0c4dee4e-4873-4f69-b4d0-c57baa642b77,2019-06-15 03:28:35,2019-06-06 04:12:25
10151,0c4dee4e-4873-4f69-b4d0-c57baa642b77,2019-06-15 03:29:07,2019-06-06 04:12:25
10152,64f367cd-4c83-47e5-bf0b-415072e30001,2019-06-18 11:54:42,2019-06-04 10:22:07


In [65]:
kaafir_trailer_episode_viewers = kaafir_trailer_episode_viewers.groupby(['viewerid']).agg({'episodestarttime':min,'trailerstarttime':min})

In [66]:
kaafir_trailer_episode_viewers = kaafir_trailer_episode_viewers.reset_index()

In [67]:
kaafir_trailer_episode_viewers

,viewerid,episodestarttime,trailerstarttime
0,0005ebdc-f74b-4750-bf09-14b038b519e8,2019-06-23 08:38:08,2019-06-23 08:33:44
1,003d07e9-2d15-4783-9677-5c2e22db56f3,2019-06-15 11:50:55,2019-06-27 07:09:35
2,006f3339-1e81-446d-83ca-d270e8708ca7,2019-06-15 04:32:09,2019-06-12 15:27:44
3,006fffee-8e6e-42b2-9d4f-ef49abe43251,2019-06-21 18:38:41,2019-06-21 15:05:07
4,00a4d03a-cdfb-4e03-b5dc-ea838c2b102f,2019-06-21 17:42:24,2019-06-04 07:05:54
...,...,...,...
1314,ff19657a-bf94-4b87-910b-90fdc42f811f,2019-06-17 13:28:24,2019-06-05 15:52:57
1315,ff3aac60-5a3d-47c3-b0a1-ca3ce27a65ee,2019-06-15 14:06:50,2019-06-10 16:50:31
1316,ff8984f8-87a2-402a-b47f-8627fff83255,2019-06-16 01:11:46,2019-06-07 14:34:22
1317,ffdf89d3-8460-486c-aead-f10d284bf50a,2019-06-16 18:48:40,2019-06-13 18:58:08


In [68]:
#checking in the episode viewers table whether the trailer start time was before the episode start time
kaafir_trailer_then_episode = kaafir_trailer_episode_viewers[kaafir_trailer_episode_viewers['trailerstarttime'] < kaafir_trailer_episode_viewers['episodestarttime'].min()]

In [69]:
kaafir_trailer_then_episode

,viewerid,episodestarttime,trailerstarttime
2,006f3339-1e81-446d-83ca-d270e8708ca7,2019-06-15 04:32:09,2019-06-12 15:27:44
4,00a4d03a-cdfb-4e03-b5dc-ea838c2b102f,2019-06-21 17:42:24,2019-06-04 07:05:54
5,00f4e7f6-9186-4404-be1d-bcfef774b76e,2019-06-14 20:54:43,2019-06-02 19:33:34
6,0107f5a3-18d1-46d0-b07c-a236a857e900,2019-06-16 07:35:29,2019-06-13 14:29:18
7,0118dd60-895d-4ef0-991f-f140391c19ef,2019-06-16 06:17:47,2019-06-07 02:32:50
...,...,...,...
1314,ff19657a-bf94-4b87-910b-90fdc42f811f,2019-06-17 13:28:24,2019-06-05 15:52:57
1315,ff3aac60-5a3d-47c3-b0a1-ca3ce27a65ee,2019-06-15 14:06:50,2019-06-10 16:50:31
1316,ff8984f8-87a2-402a-b47f-8627fff83255,2019-06-16 01:11:46,2019-06-07 14:34:22
1317,ffdf89d3-8460-486c-aead-f10d284bf50a,2019-06-16 18:48:40,2019-06-13 18:58:08


In [70]:
kaafir_trailer_then_episode = kaafir_trailer_then_episode.reset_index()

In [71]:
kaafir_trailer_then_episode_watchers = kaafir_trailer_then_episode['viewerid'].nunique()

In [72]:
print(kaafir_trailer_then_episode_watchers,"viewers of Kaafir watched the trailer first and then watched the show.")

870 viewers of Kaafir watched the trailer first and then watched the show.


<h1><b>Q12 What’s the average frequency (unique days) of platform visit for churners vs Non-churners? </b></h1>

In [73]:
#creating a separate table for churners
conviva_churners = conviva[conviva['churn'] == 1]

In [74]:
conviva_churners.shape

(403143, 23)

In [75]:
#converting the UNIX timestamps to datetime
conviva_churners['starttime'] = pd.to_datetime(conviva_churners['starttime'],unit='s')

In [76]:
conviva_churners

,viewerid,asset,deviceos,country,state,city,asn,isp,starttime,startuptime,...,averagebitrate,startuperror,ipaddress,cdn,sessiontag_c3_player_name,sessiontag_c3_client_brand,sessiontag_connectiontype,sessiontag_c3_client_hwtype,sessiontag_accesstype,churn
0,0171fdb9-d8db-4e00-a9d8-c1619811ac66,[0-0-27132] Kedarnath,Android,india,madhya pradesh,bhopal,24560,"ASN: AIRTELBROADBAND-AS-AP Bharti Airtel Ltd.,...",2019-06-03 10:25:55,5794,...,0,0,122.168.85.95,AKAMAI,Z5+FireTV+Jwplayer,Xiaomi,Data,TV,Premium,1
1,0171fdb9-d8db-4e00-a9d8-c1619811ac66,[0-0-35166] Simmba,Android,india,madhya pradesh,bhopal,24560,"ASN: AIRTELBROADBAND-AS-AP Bharti Airtel Ltd.,...",2019-06-03 10:31:20,3804,...,0,0,122.168.85.95,AMAZON,Z5+FireTV+Jwplayer,Xiaomi,Data,TV,Premium,1
2,0171fdb9-d8db-4e00-a9d8-c1619811ac66,[0-0-35021] Nirahua Hindustani 3,Android,india,madhya pradesh,indore,24560,"ASN: AIRTELBROADBAND-AS-AP Bharti Airtel Ltd.,...",2019-06-02 04:12:11,15589,...,0,0,122.170.203.36,AKAMAI,Z5+FireTV+Jwplayer,Xiaomi,Data,TV,Premium,1
3,02d7f343-f75d-4f10-a5c4-7f21ed6ff341,[0-1-215897] Skyfire - Episode 8 - The Monster,Android,india,uttar pradesh,ghaziabad,134375,ASN: FWSPL-AS-AP Fusionnet Web Services Privat...,2019-06-25 18:23:02,9716,...,0,0,103.219.230.183,AKAMAI,Z5+Android+Player_3.4.3,Oppo,WiFi,Mobile+Phone,Premium,1
4,02d7f343-f75d-4f10-a5c4-7f21ed6ff341,[0-0-6956] Great Grand Masti,Android,india,uttar pradesh,ghaziabad,134375,ASN: FWSPL-AS-AP Fusionnet Web Services Privat...,2019-06-29 18:49:09,4567,...,0,0,103.219.229.92,AKAMAI,Z5+Android+Player_3.4.3,Oppo,WiFi,Mobile+Phone,Premium,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403138,ffeccc29-f9bd-4c1d-b97f-e25c08a1f4f1,[0-1-213207] Auto Shankar - Episode 3 - The Ri...,Android,india,bihar,patna,55836,ASN: RELIANCEJIO-IN Reliance Jio Infocomm Limi...,2019-06-06 02:30:32,4417,...,0,0,47.29.28.47,AKAMAI,Z5+Android+Player_3.4.3,Oppo,Data,Mobile+Phone,Premium,1
403139,ffeccc29-f9bd-4c1d-b97f-e25c08a1f4f1,[0-0-8241] Desi Boyz,Android,india,uttar pradesh,lucknow,55836,ASN: RELIANCEJIO-IN Reliance Jio Infocomm Limi...,2019-06-03 14:15:48,5864,...,0,0,47.9.221.30,AKAMAI,Z5+Android+Player_3.4.3,Oppo,Data,Mobile+Phone,Premium,1
403140,ffeccc29-f9bd-4c1d-b97f-e25c08a1f4f1,[0-1-213126] Auto Shankar - Episode 7 - Clean ...,Android,india,bihar,patna,55836,ASN: RELIANCEJIO-IN Reliance Jio Infocomm Limi...,2019-06-06 02:11:22,2781,...,0,0,47.29.28.47,AKAMAI,Z5+Android+Player_3.4.3,Oppo,Data,Mobile+Phone,Premium,1
403141,ffeccc29-f9bd-4c1d-b97f-e25c08a1f4f1,[0-1-tvshow_1471565399-season_349665580-episod...,Android,india,bihar,patna,55836,ASN: RELIANCEJIO-IN Reliance Jio Infocomm Limi...,2019-06-06 01:30:14,2483,...,0,0,47.29.28.47,AKAMAI,Z5+Android+Player_3.4.3,Oppo,Data,Mobile+Phone,Premium,1


In [77]:
#creating a new column for storing the dates only
conviva_churners['starttimejustdate'] = conviva_churners['starttime'].dt.date

In [78]:
#calculating the unique number of dates (days) churners have visited the platform
churners_combined_days_visited = conviva_churners['starttimejustdate'].value_counts()

In [79]:
churners_total_days = churners_combined_days_visited.values.sum()

In [80]:
#umber of churners
conviva_churners['viewerid'].nunique()

62948

In [81]:
#dividing the number of days visited with number of churners get the average frequency
churners_freq_platform_visit = math.ceil(churners_total_days / conviva_churners['viewerid'].nunique())

In [82]:
print("The average frequency of platform visit for churners is",churners_freq_platform_visit,"days.")

The average frequency of platform visit for churners is 7 days.


In [83]:
#creating a separate table for non-churners
conviva_nonchurners = conviva[conviva['churn'] == 0]

In [84]:
conviva_nonchurners.shape

(645432, 23)

In [85]:
#converting the UNIX timestamps to datetime
conviva_nonchurners['starttime'] = pd.to_datetime(conviva_nonchurners['starttime'],unit='s')

In [86]:
conviva_nonchurners

,viewerid,asset,deviceos,country,state,city,asn,isp,starttime,startuptime,...,averagebitrate,startuperror,ipaddress,cdn,sessiontag_c3_player_name,sessiontag_c3_client_brand,sessiontag_connectiontype,sessiontag_c3_client_hwtype,sessiontag_accesstype,churn
403143,01005581-8076-481d-af10-b6fd1479539d,[0-1-11570] Dil Dosti Duniyadaari - Dil Dosti ...,Android,india,maharashtra,mumbai,55836,ASN: RELIANCEJIO-IN Reliance Jio Infocomm Limi...,2019-06-04 10:42:45,4076,...,0,0,49.32.52.43,AKAMAI,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0
403144,01005581-8076-481d-af10-b6fd1479539d,[0-1-11575] Dil Dosti Duniyadaari - Dil Dosti ...,Android,india,maharashtra,mumbai,17488,ASN: HATHWAY-NET-AP Hathway IP Over Cable Inte...,2019-06-03 12:02:22,1468,...,0,0,116.73.226.241,AKAMAI,Z5+Android+Player_3.4.3,Xiaomi,WiFi,Mobile+Phone,Premium,0
403145,01005581-8076-481d-af10-b6fd1479539d,[0-9-136] BTVI,Windows 10,india,maharashtra,mumbai,17488,ASN: HATHWAY-NET-AP Hathway IP Over Cable Inte...,2019-06-17 04:07:48,-1,...,0,0,115.97.20.57,AKAMAI,Z5+Web+JWPlayer+8.5.6,Google,N%2FA,Desktop,Premium,0
403146,01005581-8076-481d-af10-b6fd1479539d,[0-1-11571] Dil Dosti Duniyadaari - Dil Dosti ...,Android,india,maharashtra,mumbai,55836,ASN: RELIANCEJIO-IN Reliance Jio Infocomm Limi...,2019-06-04 10:28:37,2136,...,0,0,49.32.52.43,AKAMAI,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0
403147,01005581-8076-481d-af10-b6fd1479539d,[0-0-2494] Half Girlfriend,Android,india,maharashtra,mumbai,17488,ASN: HATHWAY-NET-AP Hathway IP Over Cable Inte...,2019-06-03 11:40:06,11487,...,0,0,116.73.226.241,AKAMAI,Z5+Android+Player_3.4.3,Xiaomi,WiFi,Mobile+Phone,Premium,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,7317642d-2608-48ff-b938-c0766b4b3447,[0-1-224908] Akka Chellellu - Akka Chellellu -...,Android,india,karnataka,gundlupet,9829,"ASN: BSNL-NIB National Internet Backbone, IN",2019-06-10 18:01:44,-1,...,0,0,117.246.121.126,TATA,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0
1048571,7317642d-2608-48ff-b938-c0766b4b3447,[0-1-224866] Ganga Manga - Manga devises a pla...,Android,india,delhi,sansad marg,9829,"ASN: BSNL-NIB National Internet Backbone, IN",2019-06-10 08:04:24,4083,...,0,0,117.209.255.97,TATA,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0
1048572,7317642d-2608-48ff-b938-c0766b4b3447,[0-1-220446] Suryavamsham - With ulterior moti...,Android,india,tamil nadu,teynampet,9829,"ASN: BSNL-NIB National Internet Backbone, IN",2019-06-01 07:23:13,3266,...,0,0,117.246.253.251,AKAMAI,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0
1048573,7317642d-2608-48ff-b938-c0766b4b3447,[0-1-230078] Muddha Mandaram - Muddha Mandaram...,Android,india,karnataka,gundlupet,9829,"ASN: BSNL-NIB National Internet Backbone, IN",2019-06-20 23:24:16,2264,...,0,0,117.249.225.150,AMAZON,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0


In [87]:
#creating a new column for storing the dates only
conviva_nonchurners['starttimejustdate'] = conviva_nonchurners['starttime'].dt.date

In [88]:
#calculating the unique number of dates (days) non-churners have visited the platform
nonchurners_combined_days_visited = conviva_nonchurners['starttimejustdate'].value_counts()

In [89]:
nonchurners_total_days = nonchurners_combined_days_visited.values.sum()

In [90]:
#number of non-churners
conviva_nonchurners['viewerid'].nunique()

37584

In [91]:
#dividing the number of days visited with number of non-churners to get the average frequency
nonchurners_freq_platform_visit = math.ceil(nonchurners_total_days / conviva_nonchurners['viewerid'].nunique())

In [92]:
print("The average frequency of platform visit for non-churners is",nonchurners_freq_platform_visit,"days.")

The average frequency of platform visit for non-churners is 18 days.


<h1></b>Q13 Top 10 shows watched by churners & non churners?</b></h1>

In [93]:
churners_shows = conviva_churners['asset'].value_counts()

In [94]:
#top 10 shows watched by churners
churners_shows.nlargest(10)

[0-0-39320] The Accidental Prime Minister              14092
[0-0-33204] Uri: The Surgical Strike                   12716
[0-0-35166] Simmba                                      9337
[0-0-27132] Kedarnath                                   8524
[0-1-226358] Kaafir - Episode 1 - The First Meeting     7692
[0-0-47046] Why Cheat India                             5391
[0-0-movie_1424717826] Veere Di Wedding                 4100
[0-1-215883] Skyfire - Skyfire - Episode 1 - Pralay     3852
[0-0-6728] Mulk                                         2781
[0-0-59341] Jersey                                      2700
Name: asset, dtype: int64

In [95]:
nonchurners_shows = conviva_nonchurners['asset'].value_counts()

In [96]:
#top 10 shows watched by churners
nonchurners_shows.nlargest(10)

[0-1-226358] Kaafir - Episode 1 - The First Meeting    11213
[0-0-33204] Uri: The Surgical Strike                    9455
[0-0-39320] The Accidental Prime Minister               9398
[0-0-35166] Simmba                                      7038
[0-0-27132] Kedarnath                                   6257
[0-0-59341] Jersey                                      6020
[0-0-47046] Why Cheat India                             4506
[0-9-zeemarathi] Zee Marathi                            4410
[0-1-215883] Skyfire - Skyfire - Episode 1 - Pralay     3786
[0-0-61493] ABCD                                        3307
Name: asset, dtype: int64

<h1><b>Q14 What is the average engagement (watch time) of churn and non-churn users on weekdays and weekends? </b></h1>

In [97]:
#adding the calculated weekday column to the non-churners table
conviva_nonchurners['weekday'] = conviva_nonchurners['starttime'].dt.weekday

In [98]:
#selecting only those entries who have datetime entries of weekdays
conviva_nonchurners_weekdays = conviva_nonchurners[conviva_nonchurners['weekday'] < 5]

In [99]:
conviva_nonchurners_weekdays.shape

(443408, 25)

In [100]:
#calculating the total engagement time on weekdays
nonchurners_playingtime_weekday = conviva_nonchurners_weekdays['playingtime'].sum()

In [101]:
#calculating the average by dividing the total weekday engagement time with number of weekday entries
nonchurners_avg_playingtime_weekday = nonchurners_playingtime_weekday / len(conviva_nonchurners_weekdays)

In [102]:
print("The average engagement time of non-churners on weekdays is",math.ceil(nonchurners_avg_playingtime_weekday / 60000),"minutes.")

The average engagement time of non-churners on weekdays is 9 minutes.


In [103]:
#selecting only those entries who have datetime entries of weekdends
conviva_nonchurners_weekends = conviva_nonchurners[conviva_nonchurners['weekday'] > 4]

In [104]:
conviva_nonchurners_weekends.shape

(202024, 25)

In [105]:
#calculating the total engagement time on weekends
nonchurners_playingtime_weekends = conviva_nonchurners_weekends['playingtime'].sum()

In [106]:
#calculating the average by dividing the total weekend engagement time with number of weekend entries
nonchurners_avg_playingtime_weekends = nonchurners_playingtime_weekends / len(conviva_nonchurners_weekends)

In [107]:
print("The average engagement time of non-churners on weekends is",math.ceil(nonchurners_avg_playingtime_weekends / 60000),"minutes.")

The average engagement time of non-churners on weekends is 11 minutes.


In [108]:
#adding the calculated weekday column to the churners table
conviva_churners['weekday'] = conviva_churners['starttime'].dt.weekday

In [109]:
#selecting only those entries who have datetime entries of weekdays
conviva_churners_weekdays = conviva_churners[conviva_churners['weekday'] < 5]

In [110]:
conviva_churners_weekdays.shape

(260133, 25)

In [111]:
#calculating the total engagement time on weekdays
churners_playingtime_weekdays = conviva_churners_weekdays['playingtime'].sum()

In [112]:
#calculating the average by dividing the total weekday engagement time with number of weekday entries
churners_avg_playingtime_weekdays = churners_playingtime_weekdays / len(conviva_churners_weekdays)

In [113]:
print("The average engagement time of churners on weekdays is",math.ceil(churners_avg_playingtime_weekdays / 60000),"minutes.")

The average engagement time of churners on weekdays is 10 minutes.


In [114]:
#selecting only those entries who have datetime entries of weekends
conviva_churners_weekends = conviva_churners[conviva_churners['weekday'] > 4]

In [115]:
conviva_churners_weekends.shape

(143010, 25)

In [116]:
#calculating the total engagement time on weekends
churners_playingtime_weekends = conviva_churners_weekends['playingtime'].sum()

In [117]:
#calculating the average by dividing the total weekday engagement time with number of weekday entries
churners_avg_playingtime_weekends = churners_playingtime_weekends / len(conviva_churners_weekends)

In [118]:
print("The average engagement time of churners on weekends is",math.ceil(churners_avg_playingtime_weekends / 60000),"minutes.")

The average engagement time of churners on weekends is 12 minutes.


<h1><b>Q7 How many watched All Episodes of Kaafir and at least 5 minutes of each episodes?</b></h1>

In [119]:
#selecting not null entries from conviva which have Kaafir - Episode as the asset
kaafir = conviva_notna[conviva_notna['asset'].str.contains('Kaafir - Episode')]

In [120]:
#calculated total viewing time per user per asset, i.e per episode
kaafir_unique_episode_records = kaafir.groupby(['viewerid','asset'])['playingtime'].sum()

In [121]:
kaafir_unique_episode_records

viewerid                              asset                                                     
0005ebdc-f74b-4750-bf09-14b038b519e8  [0-1-226358] Kaafir - Episode 1 - The First Meeting           1621961
                                      [0-1-226365] Kaafir - Episode 2 - A Tale of Retrospection        1262
00084ff7-d0f8-40dd-8221-9a10865408c7  [0-1-226358] Kaafir - Episode 1 - The First Meeting           1636099
                                      [0-1-226367] Kaafir - Episode 3 - Truth & Lies                      0
                                      [0-1-226377] Kaafir - Episode 5 - Redemption & Retribution    1898462
                                                                                                     ...   
fffbe4f0-9ace-4323-afae-3abc9b224a3d  [0-1-226377] Kaafir - Episode 5 - Redemption & Retribution     117280
fffc55a4-06b8-488b-92cd-1940486f95d9  [0-1-226358] Kaafir - Episode 1 - The First Meeting           1083533
fffe8b4b-a687-4afb-8d5f-b200901e2fdc  [

In [122]:
#gave the columns their original names
kaafir_unique_episode_records = kaafir_unique_episode_records.reset_index()

In [123]:
#selected only those entries who have viewing time more than 5 minutes
kaafir_allepisodes_atleast5min = kaafir_unique_episode_records[kaafir_unique_episode_records['playingtime'] >= 30000]

In [124]:
kaafir_allepisodes_atleast5min.shape

(21546, 3)

In [125]:
#lsit of those viewers who have watched at least 5 minutes of an episode
grthan5min_viewerid_list = list(kaafir_allepisodes_atleast5min['viewerid'])

In [126]:
#list of those viewers who have watched all episodes
kaafir_all_episode_watchers = list(kaafir_all_watchers.index)

In [127]:
#intersection of the above 2 lists will give us those viewers who have watched all episodes and at least 5 minutes of each episode
kaafir_grthan5min_allepisodes_watchers = list(set(grthan5min_viewerid_list) & set(kaafir_all_episode_watchers))

In [128]:
print("The number of viewers who watched all episodes of Kaafir and at least 5 minutes of each episodes is",len(kaafir_grthan5min_allepisodes_watchers))

The number of viewers who watched all episodes of Kaafir and at least 5 minutes of each episodes is 129


<h1><b>Q6 How many watched All episodes of Kaafir (irrespective of how much) within 24 hours from when they started? </b></h1>

In [129]:
#created a new table grouped by viewerid and selected the first start time and total engagement time
kaafir_within_24hr = kaafir.groupby(['viewerid']).agg({'starttime':min,'playingtime':sum})

In [130]:
#gave the columns their original names
kaafir_within_24hr = kaafir_within_24hr.reset_index()

In [131]:
kaafir_within_24hr.shape

(11793, 3)

In [132]:
#calculated the end of viewing time by adding the start time to the total engagement time
kaafir_within_24hr['endtime'] = kaafir_within_24hr['starttime'] + kaafir_within_24hr['playingtime']

In [133]:
kaafir_within_24hr

,viewerid,starttime,playingtime,endtime
0,0005ebdc-f74b-4750-bf09-14b038b519e8,1561279088,1623223,1562902311
1,00084ff7-d0f8-40dd-8221-9a10865408c7,1561054745,6055371,1567110116
2,0008badf-5d15-4106-982c-1e068b47f48a,1561831631,21826,1561853457
3,000f686c-b5f5-45dd-9fa5-02eb8a24c30c,1561646240,3170298,1564816538
4,00228901-fc03-43ea-aac6-59db63c55bd2,1560937802,414458,1561352260
...,...,...,...,...
11788,ffe8cf49-a0f0-4c21-9b3f-a9e3add69f20,1560618253,2097727,1562715980
11789,ffe9f7a1-d872-41cd-b337-c26a6f0904e4,1561613982,0,1561613982
11790,fffbe4f0-9ace-4323-afae-3abc9b224a3d,1560873643,2242091,1563115734
11791,fffc55a4-06b8-488b-92cd-1940486f95d9,1561001096,1083533,1562084629


In [134]:
#converted the UNIX timestamps to datetime format
kaafir_within_24hr['starttime'] = pd.to_datetime(kaafir_within_24hr['starttime'],unit='s')

In [135]:
#converted the UNIX timestamps to datetime format
kaafir_within_24hr['endtime'] = pd.to_datetime(kaafir_within_24hr['endtime'],unit='s')

In [136]:
kaafir_within_24hr

,viewerid,starttime,playingtime,endtime
0,0005ebdc-f74b-4750-bf09-14b038b519e8,2019-06-23 08:38:08,1623223,2019-07-12 03:31:51
1,00084ff7-d0f8-40dd-8221-9a10865408c7,2019-06-20 18:19:05,6055371,2019-08-29 20:21:56
2,0008badf-5d15-4106-982c-1e068b47f48a,2019-06-29 18:07:11,21826,2019-06-30 00:10:57
3,000f686c-b5f5-45dd-9fa5-02eb8a24c30c,2019-06-27 14:37:20,3170298,2019-08-03 07:15:38
4,00228901-fc03-43ea-aac6-59db63c55bd2,2019-06-19 09:50:02,414458,2019-06-24 04:57:40
...,...,...,...,...
11788,ffe8cf49-a0f0-4c21-9b3f-a9e3add69f20,2019-06-15 17:04:13,2097727,2019-07-09 23:46:20
11789,ffe9f7a1-d872-41cd-b337-c26a6f0904e4,2019-06-27 05:39:42,0,2019-06-27 05:39:42
11790,fffbe4f0-9ace-4323-afae-3abc9b224a3d,2019-06-18 16:00:43,2242091,2019-07-14 14:48:54
11791,fffc55a4-06b8-488b-92cd-1940486f95d9,2019-06-20 03:24:56,1083533,2019-07-02 16:23:49


In [137]:
#selected only those entries where the difference between the start time and end time is less than or equal to 24 hours
kaafir_within_24hr = kaafir_within_24hr[(kaafir_within_24hr['endtime'] - kaafir_within_24hr['starttime']).astype('timedelta64[h]') <=24]

In [138]:
kaafir_within_24hr.shape

(4345, 4)

In [139]:
#collected all viewerids in a list
kaafir_within_24hr_viewers = list(kaafir_within_24hr['viewerid'])

In [140]:
#performed intersection of the list with all Kaafir episode viewers and the list with all Kaafir viewers who stopped viewing within 24 hours
kaafir_allepisodes_24hr = list(set(kaafir_all_episode_watchers) & set(kaafir_within_24hr_viewers))

In [141]:
kaafir_allepisodes_24hr

['e995de0a-dcc0-4cb6-b60c-1e48cc355b2e',
 '93640199-bca2-402a-9cc6-97bb06d63517',
 '369d3f49-dc0f-4a82-b6f8-f55f7f3f6d14']

In [142]:
print(len(kaafir_allepisodes_24hr),"viewers have watched all episodes of Kaafir irrespective of viewing time in 24 hours from when they started.")

3 viewers have watched all episodes of Kaafir irrespective of viewing time in 24 hours from when they started.


<h1><b>Q8 Those who have watched more than 50% duration of the kaafir, what else they have watched? (Mention Top 10 in order – top most being the most watched)</b></h1> 

In [143]:
#conviva table with not null asset entries
conviva_notna

,viewerid,asset,deviceos,country,state,city,asn,isp,starttime,startuptime,...,averagebitrate,startuperror,ipaddress,cdn,sessiontag_c3_player_name,sessiontag_c3_client_brand,sessiontag_connectiontype,sessiontag_c3_client_hwtype,sessiontag_accesstype,churn
0,0171fdb9-d8db-4e00-a9d8-c1619811ac66,[0-0-27132] Kedarnath,Android,india,madhya pradesh,bhopal,24560,"ASN: AIRTELBROADBAND-AS-AP Bharti Airtel Ltd.,...",1559557555,5794,...,0,0,122.168.85.95,AKAMAI,Z5+FireTV+Jwplayer,Xiaomi,Data,TV,Premium,1
1,0171fdb9-d8db-4e00-a9d8-c1619811ac66,[0-0-35166] Simmba,Android,india,madhya pradesh,bhopal,24560,"ASN: AIRTELBROADBAND-AS-AP Bharti Airtel Ltd.,...",1559557880,3804,...,0,0,122.168.85.95,AMAZON,Z5+FireTV+Jwplayer,Xiaomi,Data,TV,Premium,1
2,0171fdb9-d8db-4e00-a9d8-c1619811ac66,[0-0-35021] Nirahua Hindustani 3,Android,india,madhya pradesh,indore,24560,"ASN: AIRTELBROADBAND-AS-AP Bharti Airtel Ltd.,...",1559448731,15589,...,0,0,122.170.203.36,AKAMAI,Z5+FireTV+Jwplayer,Xiaomi,Data,TV,Premium,1
3,02d7f343-f75d-4f10-a5c4-7f21ed6ff341,[0-1-215897] Skyfire - Episode 8 - The Monster,Android,india,uttar pradesh,ghaziabad,134375,ASN: FWSPL-AS-AP Fusionnet Web Services Privat...,1561486982,9716,...,0,0,103.219.230.183,AKAMAI,Z5+Android+Player_3.4.3,Oppo,WiFi,Mobile+Phone,Premium,1
4,02d7f343-f75d-4f10-a5c4-7f21ed6ff341,[0-0-6956] Great Grand Masti,Android,india,uttar pradesh,ghaziabad,134375,ASN: FWSPL-AS-AP Fusionnet Web Services Privat...,1561834149,4567,...,0,0,103.219.229.92,AKAMAI,Z5+Android+Player_3.4.3,Oppo,WiFi,Mobile+Phone,Premium,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,7317642d-2608-48ff-b938-c0766b4b3447,[0-1-224908] Akka Chellellu - Akka Chellellu -...,Android,india,karnataka,gundlupet,9829,"ASN: BSNL-NIB National Internet Backbone, IN",1560189704,-1,...,0,0,117.246.121.126,TATA,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0
1048571,7317642d-2608-48ff-b938-c0766b4b3447,[0-1-224866] Ganga Manga - Manga devises a pla...,Android,india,delhi,sansad marg,9829,"ASN: BSNL-NIB National Internet Backbone, IN",1560153864,4083,...,0,0,117.209.255.97,TATA,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0
1048572,7317642d-2608-48ff-b938-c0766b4b3447,[0-1-220446] Suryavamsham - With ulterior moti...,Android,india,tamil nadu,teynampet,9829,"ASN: BSNL-NIB National Internet Backbone, IN",1559373793,3266,...,0,0,117.246.253.251,AKAMAI,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0
1048573,7317642d-2608-48ff-b938-c0766b4b3447,[0-1-230078] Muddha Mandaram - Muddha Mandaram...,Android,india,karnataka,gundlupet,9829,"ASN: BSNL-NIB National Internet Backbone, IN",1561073056,2264,...,0,0,117.249.225.150,AMAZON,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0


In [144]:
#table of all entries with asset as "Kaafir"
kaafir_viewers

,viewerid,asset,deviceos,country,state,city,asn,isp,starttime,startuptime,...,startuperror,ipaddress,cdn,sessiontag_c3_player_name,sessiontag_c3_client_brand,sessiontag_connectiontype,sessiontag_c3_client_hwtype,sessiontag_accesstype,churn,starttimejustdate
41,0a1e752b-bac6-4b06-8bfd-e7a41421cdc8,[0-1-226371] Kaafir - Episode 4 - Judgement Day,Android,india,karnataka,bengaluru,45609,Bharti Airtel,2019-06-15 18:37:24,2254,...,0,117.97.225.63,AMAZON,Z5+Android+Player_3.4.3,OnePlus,Data,Mobile+Phone,Premium,1,2019-06-15
43,0a1e752b-bac6-4b06-8bfd-e7a41421cdc8,[0-1-226388] Kaafir - Episode 6 - Where Do We ...,Android,india,karnataka,bengaluru,45609,Bharti Airtel,2019-06-15 21:01:18,1585,...,0,117.97.225.63,AKAMAI,Z5+Android+Player_3.4.3,OnePlus,Data,Mobile+Phone,Premium,1,2019-06-15
45,0a1e752b-bac6-4b06-8bfd-e7a41421cdc8,[0-1-226358] Kaafir - Episode 1 - The First Me...,Android,india,karnataka,bengaluru,45609,Bharti Airtel,2019-06-15 19:03:13,-1,...,0,117.97.225.63,TATA,Z5+Android+Player_3.4.3,OnePlus,Data,Mobile+Phone,Premium,1,2019-06-15
46,0a1e752b-bac6-4b06-8bfd-e7a41421cdc8,[0-1-226371] Kaafir - Episode 4 - Judgement Day,Android,india,karnataka,bengaluru,45609,Bharti Airtel,2019-06-15 18:37:24,2254,...,0,117.97.225.63,AMAZON,Z5+Android+Player_3.4.3,OnePlus,Data,Mobile+Phone,Premium,1,2019-06-15
47,0a1e752b-bac6-4b06-8bfd-e7a41421cdc8,[0-1-226358] Kaafir - Episode 1 - The First Me...,Android,india,karnataka,bengaluru,45609,Bharti Airtel,2019-06-15 21:01:10,1703,...,0,117.97.225.63,AKAMAI,Z5+Android+Player_3.4.3,OnePlus,Data,Mobile+Phone,Premium,1,2019-06-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048326,718534a6-6b53-4e7d-8e04-f25f346744b1,[0-1-226371] Kaafir - Episode 4 - Judgement Day,Fire OS,india,maharashtra,mumbai,55836,ASN: RELIANCEJIO-IN Reliance Jio Infocomm Limi...,2019-06-16 15:17:21,2631,...,0,49.33.250.48,TATA,Z5+FireTV+Jwplayer,Amazon,WiFi,Set+Top+Box,Premium,0,2019-06-16
1048329,718534a6-6b53-4e7d-8e04-f25f346744b1,[0-1-226388] Kaafir - Episode 6 - Where Do We ...,Fire OS,india,maharashtra,prabhadevi,38266,Vodafone,2019-06-16 17:37:36,8566,...,0,42.106.217.29,AKAMAI,Z5+FireTV+Jwplayer,Amazon,WiFi,Set+Top+Box,Premium,0,2019-06-16
1048330,718534a6-6b53-4e7d-8e04-f25f346744b1,[0-1-226388] Kaafir - Episode 6 - Where Do We ...,Fire OS,india,maharashtra,prabhadevi,38266,Vodafone,2019-06-16 17:09:55,3561,...,0,42.106.217.96,AKAMAI,Z5+FireTV+Jwplayer,Amazon,WiFi,Set+Top+Box,Premium,0,2019-06-16
1048332,718534a6-6b53-4e7d-8e04-f25f346744b1,[0-1-226371] Kaafir - Episode 4 - Judgement Day,Fire OS,india,maharashtra,prabhadevi,38266,Vodafone,2019-06-16 15:30:16,2763,...,0,42.106.217.96,AKAMAI,Z5+FireTV+Jwplayer,Amazon,WiFi,Set+Top+Box,Premium,0,2019-06-16


<h1><b>Q15 Does Video Start time impact churn?</b></h1>

In [145]:
#performing Pearson's correlation test to find out whether video start time impacts churn
stat, p = pearsonr(conviva_notna['startuptime'], conviva_notna['churn'])
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
    print('Video start time probably does not impact churn')
else:
    print('Video start time probably impacts churn')

stat=0.021, p=0.000
Video start time probably impacts churn


<h1><b>Q16 Does Rebuffering time impact churn?</b></h1>

In [146]:
#performing Pearson's correlation test to find out whether buffering time impacts churn
stat, p = pearsonr(conviva_notna['bufferingtime'], conviva_notna['churn'])
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
    print('Rebuffering time probably does not impact churn')
else:
    print('Rebuffering time probably impacts churn')

stat=0.008, p=0.000
Rebuffering time probably impacts churn


<h1><b>Modelling exercise</b></h1>

In [147]:
conviva_notna

,viewerid,asset,deviceos,country,state,city,asn,isp,starttime,startuptime,...,averagebitrate,startuperror,ipaddress,cdn,sessiontag_c3_player_name,sessiontag_c3_client_brand,sessiontag_connectiontype,sessiontag_c3_client_hwtype,sessiontag_accesstype,churn
0,0171fdb9-d8db-4e00-a9d8-c1619811ac66,[0-0-27132] Kedarnath,Android,india,madhya pradesh,bhopal,24560,"ASN: AIRTELBROADBAND-AS-AP Bharti Airtel Ltd.,...",1559557555,5794,...,0,0,122.168.85.95,AKAMAI,Z5+FireTV+Jwplayer,Xiaomi,Data,TV,Premium,1
1,0171fdb9-d8db-4e00-a9d8-c1619811ac66,[0-0-35166] Simmba,Android,india,madhya pradesh,bhopal,24560,"ASN: AIRTELBROADBAND-AS-AP Bharti Airtel Ltd.,...",1559557880,3804,...,0,0,122.168.85.95,AMAZON,Z5+FireTV+Jwplayer,Xiaomi,Data,TV,Premium,1
2,0171fdb9-d8db-4e00-a9d8-c1619811ac66,[0-0-35021] Nirahua Hindustani 3,Android,india,madhya pradesh,indore,24560,"ASN: AIRTELBROADBAND-AS-AP Bharti Airtel Ltd.,...",1559448731,15589,...,0,0,122.170.203.36,AKAMAI,Z5+FireTV+Jwplayer,Xiaomi,Data,TV,Premium,1
3,02d7f343-f75d-4f10-a5c4-7f21ed6ff341,[0-1-215897] Skyfire - Episode 8 - The Monster,Android,india,uttar pradesh,ghaziabad,134375,ASN: FWSPL-AS-AP Fusionnet Web Services Privat...,1561486982,9716,...,0,0,103.219.230.183,AKAMAI,Z5+Android+Player_3.4.3,Oppo,WiFi,Mobile+Phone,Premium,1
4,02d7f343-f75d-4f10-a5c4-7f21ed6ff341,[0-0-6956] Great Grand Masti,Android,india,uttar pradesh,ghaziabad,134375,ASN: FWSPL-AS-AP Fusionnet Web Services Privat...,1561834149,4567,...,0,0,103.219.229.92,AKAMAI,Z5+Android+Player_3.4.3,Oppo,WiFi,Mobile+Phone,Premium,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,7317642d-2608-48ff-b938-c0766b4b3447,[0-1-224908] Akka Chellellu - Akka Chellellu -...,Android,india,karnataka,gundlupet,9829,"ASN: BSNL-NIB National Internet Backbone, IN",1560189704,-1,...,0,0,117.246.121.126,TATA,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0
1048571,7317642d-2608-48ff-b938-c0766b4b3447,[0-1-224866] Ganga Manga - Manga devises a pla...,Android,india,delhi,sansad marg,9829,"ASN: BSNL-NIB National Internet Backbone, IN",1560153864,4083,...,0,0,117.209.255.97,TATA,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0
1048572,7317642d-2608-48ff-b938-c0766b4b3447,[0-1-220446] Suryavamsham - With ulterior moti...,Android,india,tamil nadu,teynampet,9829,"ASN: BSNL-NIB National Internet Backbone, IN",1559373793,3266,...,0,0,117.246.253.251,AKAMAI,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0
1048573,7317642d-2608-48ff-b938-c0766b4b3447,[0-1-230078] Muddha Mandaram - Muddha Mandaram...,Android,india,karnataka,gundlupet,9829,"ASN: BSNL-NIB National Internet Backbone, IN",1561073056,2264,...,0,0,117.249.225.150,AMAZON,Z5+Android+Player_3.4.3,Xiaomi,Data,Mobile+Phone,Premium,0


In [148]:
conviva_notna.isna().sum()

viewerid                           0
asset                              0
deviceos                          12
country                            0
state                              0
city                               0
asn                                0
isp                                0
starttime                          0
startuptime                        0
playingtime                        0
bufferingtime                      0
interrupts                         0
averagebitrate                     0
startuperror                       0
ipaddress                          0
cdn                                0
sessiontag_c3_player_name          0
sessiontag_c3_client_brand     15909
sessiontag_connectiontype        200
sessiontag_c3_client_hwtype     2012
sessiontag_accesstype              0
churn                              0
dtype: int64

In [149]:
conviva_notna = conviva_notna.dropna()

In [150]:
conviva_notna.isna().sum()

viewerid                       0
asset                          0
deviceos                       0
country                        0
state                          0
city                           0
asn                            0
isp                            0
starttime                      0
startuptime                    0
playingtime                    0
bufferingtime                  0
interrupts                     0
averagebitrate                 0
startuperror                   0
ipaddress                      0
cdn                            0
sessiontag_c3_player_name      0
sessiontag_c3_client_brand     0
sessiontag_connectiontype      0
sessiontag_c3_client_hwtype    0
sessiontag_accesstype          0
churn                          0
dtype: int64

In [151]:
conviva_notna.shape

(1030937, 23)

In [152]:
model_data = conviva_notna.groupby(['viewerid','churn']).agg({'asset':'unique',
                                                    'sessiontag_c3_player_name':'unique',
                                                    'sessiontag_c3_client_brand':'unique',
                                                    'sessiontag_connectiontype':'unique',
                                                    'sessiontag_c3_client_hwtype':'unique',
                                                    'starttime':'unique',
                                                    'startuptime':'mean',
                                                    'playingtime':'mean',
                                                    'bufferingtime':'mean',
                                                   'interrupts':'mean',
                                                   'startuperror':'mean'
                                                    })

In [153]:
model_data

,,asset,sessiontag_c3_player_name,sessiontag_c3_client_brand,sessiontag_connectiontype,sessiontag_c3_client_hwtype,starttime,startuptime,playingtime,bufferingtime,interrupts,startuperror
viewerid,churn,,,,,,,,,,,
00004c28-8e4d-42b9-89bd-76f8821b6d15,1,[[0-1-222011] Kailasapuram - Episode 1 - Caugh...,[Z5+Android+Player_3.4.3],[OnePlus],[Data],[Mobile+Phone],"[1559993798, 1560879936]",-1.000000,0.000000e+00,0.000000,0.000000,0.000000
00008f8d-cb75-4048-9f4c-21995eba4215,1,[[0-0-movie_1152700001] Saheb Biwi Aur Gangste...,[Z5+FireTV+Jwplayer],[Xiaomi],[WiFi],[TV],"[1561744062, 1561477500, 1561389579, 156120915...",6721.000000,1.521220e+06,41198.360000,9.400000,0.000000
0000d90b-0f4e-4535-91ed-cbbc91c18cc4,1,"[[0-0-47046] Why Cheat India, [0-0-35166] Simm...","[Z5+FireTV+Jwplayer, Z5+Android+Player_3.4.3]","[Xiaomi, Oppo]",[WiFi],"[TV, Mobile+Phone]","[1559408606, 1560589229, 1560588546, 156104570...",2205.666667,4.223742e+05,20434.833333,40.500000,0.000000
0002a036-a05a-42cf-9941-0c557967a92c,1,"[[0-0-27132] Kedarnath, [0-0-19151] Dhadak, [0...","[Z5+Android+Player_3.4.3, Z5+Web+JWPlayer+8.5.6]","[Samsung, Google]","[Data, N%2FA]","[Mobile+Phone, Desktop]","[1559762680, 1559761812, 1559934558, 155992769...",1678.800000,1.038950e+06,177826.800000,14.200000,0.000000
0003fbeb-b802-413a-b576-1391d1913e40,1,[[0-0-47046] Why Cheat India],[Z5+Android+Player_3.4.3],[Samsung],[Data],[Mobile+Phone],"[1561544422, 1561298118]",2967.000000,1.747370e+05,1243.500000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
fffd58bf-87b1-4497-a882-39d9f0902e87,1,[[0-1-223870] Kundali Bhagya - Srishti learns ...,[Z5+Android+Player_3.4.3],[Comneed],[Data],[Mobile+Phone],"[1559932719, 1559932858, 1560246736]",1760.000000,5.309150e+05,20632.666667,6.333333,0.000000
fffe427d-8caa-4720-94d9-77a73cfe3c8f,0,[[0-1-202652] Poison - Episode 1 - The Hunter'...,[Z5+Android+Player_3.4.3],[Xiaomi],"[WiFi, Data]",[Mobile+Phone],"[1560240920, 1561098888, 1560157751, 156084580...",1942.478261,3.662239e+04,5165.565217,1.000000,0.000000
fffe8b4b-a687-4afb-8d5f-b200901e2fdc,1,[[0-0-newsauto_14j8ksep9djg] Chala Hawa Yeu Dy...,[Z5+Android+Player_3.4.3],"[Motorola, Samsung]","[Data, WiFi]","[Mobile+Phone, Tablet]","[1560160110, 1559485314, 1559487116, 155974557...",2073.410256,7.192786e+05,2697.333333,1.487179,0.000000


In [154]:
type(model_data)

pandas.core.frame.DataFrame

In [155]:
model_data = model_data.reset_index()

In [156]:
model_data

,viewerid,churn,asset,sessiontag_c3_player_name,sessiontag_c3_client_brand,sessiontag_connectiontype,sessiontag_c3_client_hwtype,starttime,startuptime,playingtime,bufferingtime,interrupts,startuperror
0,00004c28-8e4d-42b9-89bd-76f8821b6d15,1,[[0-1-222011] Kailasapuram - Episode 1 - Caugh...,[Z5+Android+Player_3.4.3],[OnePlus],[Data],[Mobile+Phone],"[1559993798, 1560879936]",-1.000000,0.000000e+00,0.000000,0.000000,0.000000
1,00008f8d-cb75-4048-9f4c-21995eba4215,1,[[0-0-movie_1152700001] Saheb Biwi Aur Gangste...,[Z5+FireTV+Jwplayer],[Xiaomi],[WiFi],[TV],"[1561744062, 1561477500, 1561389579, 156120915...",6721.000000,1.521220e+06,41198.360000,9.400000,0.000000
2,0000d90b-0f4e-4535-91ed-cbbc91c18cc4,1,"[[0-0-47046] Why Cheat India, [0-0-35166] Simm...","[Z5+FireTV+Jwplayer, Z5+Android+Player_3.4.3]","[Xiaomi, Oppo]",[WiFi],"[TV, Mobile+Phone]","[1559408606, 1560589229, 1560588546, 156104570...",2205.666667,4.223742e+05,20434.833333,40.500000,0.000000
3,0002a036-a05a-42cf-9941-0c557967a92c,1,"[[0-0-27132] Kedarnath, [0-0-19151] Dhadak, [0...","[Z5+Android+Player_3.4.3, Z5+Web+JWPlayer+8.5.6]","[Samsung, Google]","[Data, N%2FA]","[Mobile+Phone, Desktop]","[1559762680, 1559761812, 1559934558, 155992769...",1678.800000,1.038950e+06,177826.800000,14.200000,0.000000
4,0003fbeb-b802-413a-b576-1391d1913e40,1,[[0-0-47046] Why Cheat India],[Z5+Android+Player_3.4.3],[Samsung],[Data],[Mobile+Phone],"[1561544422, 1561298118]",2967.000000,1.747370e+05,1243.500000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99312,fffd58bf-87b1-4497-a882-39d9f0902e87,1,[[0-1-223870] Kundali Bhagya - Srishti learns ...,[Z5+Android+Player_3.4.3],[Comneed],[Data],[Mobile+Phone],"[1559932719, 1559932858, 1560246736]",1760.000000,5.309150e+05,20632.666667,6.333333,0.000000
99313,fffe427d-8caa-4720-94d9-77a73cfe3c8f,0,[[0-1-202652] Poison - Episode 1 - The Hunter'...,[Z5+Android+Player_3.4.3],[Xiaomi],"[WiFi, Data]",[Mobile+Phone],"[1560240920, 1561098888, 1560157751, 156084580...",1942.478261,3.662239e+04,5165.565217,1.000000,0.000000
99314,fffe8b4b-a687-4afb-8d5f-b200901e2fdc,1,[[0-0-newsauto_14j8ksep9djg] Chala Hawa Yeu Dy...,[Z5+Android+Player_3.4.3],"[Motorola, Samsung]","[Data, WiFi]","[Mobile+Phone, Tablet]","[1560160110, 1559485314, 1559487116, 155974557...",2073.410256,7.192786e+05,2697.333333,1.487179,0.000000
99315,ffff6f1d-490b-4895-92e4-16b2e3bc3d9b,0,[[0-1-230270] - Paakhi saves Natasha from the ...,"[Zee5+iOS+JWPlayer+3.1.3%2B73, Zee5+iOS+JWPlay...","[Zen, Apple]",[WiFi],[Mobile+Phone],"[1561093278, 1560948935, 1559653705, 155992878...",4711.183673,2.613446e+05,50081.734694,14.714286,0.040816


In [157]:
model_data.shape

(99317, 13)

In [158]:
model_data['sessiontag_c3_player_name'][0][0]

'Z5+Android+Player_3.4.3'